In [76]:
import pandas as pd
import numpy as np
import pymssql #pyodbc
from openpyxl import Workbook
from openpyxl import load_workbook

# static parameters

current_date = '20210331'
path= 'simulation.xlsx'

r = 500   # simulation row
c = 2000  # simulation column

k = 1.4 # coefficient [1:2]

In [77]:
# generate two random rows

A = np.random.randint(r*k, size=(2, c))

In [78]:
# generate new simulation zero data matrix

M = np.zeros([r-2, c])

In [79]:
# merge A & M random matrices

N = np.vstack((A, M))
# M.shape

In [80]:
# generate random numbers on each row

for i in range(2, r):

    a = np.minimum(N[i-2,:], N[i-1,:])
    b = np.maximum(N[i-2,:], N[i-1,:])
    
    N[i,:] = np.random.randint(a-k*10, b+k*10)
    

In [81]:
# copy template file and append data into this file

wb = load_workbook(path)
ws = wb['template']

ws["C7"]  = r # Generated rows
ws["C8"]  = c # Generated columns
ws["C9"]  = np.min(N) # Min. value
ws["C10"] = np.max(N) # Max. value
ws["C11"] = np.mean(N) # Mean
ws["C12"] = np.std(N) # Std dev



new_name = 'Simulation-'+current_date+'.xlsx'
wb.save(new_name)
wb.close()

In [82]:
# append dataframe to excel

writer = pd.ExcelWriter(new_name, engine = 'openpyxl')
writer.book = load_workbook(new_name)

writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)

pd.DataFrame(N).T.to_excel(writer, sheet_name='simulation data',
             startrow=0, startcol=0, header=False, index=False)


writer.save()
writer.close()
